# Anna KaRNNa

In this notebook, we'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see the characters encoded as integers.

In [4]:
encoded[:100]

array([44, 72, 37, 12, 25, 41, 38, 54, 47, 52, 52, 52, 74, 37, 12, 12, 55,
       54, 45, 37,  8,  0, 40,  0, 41, 78, 54, 37, 38, 41, 54, 37, 40, 40,
       54, 37, 40,  0,  5, 41,  9, 54, 41, 77, 41, 38, 55, 54, 28, 35, 72,
       37, 12, 12, 55, 54, 45, 37,  8,  0, 40, 55, 54,  0, 78, 54, 28, 35,
       72, 37, 12, 12, 55, 54,  0, 35, 54,  0, 25, 78, 54, 22, 14, 35, 52,
       14, 37, 55,  2, 52, 52, 42, 77, 41, 38, 55, 25, 72,  0, 35])

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [5]:
len(vocab)

83

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>
We have our text encoded as integers as one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the number of batches we can make from some array `arr`, you divide the length of `arr` by the batch size. Once you know the number of batches and the batch size, you can get the total number of characters to keep.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`n_seqs` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$ where $K$ is the number of batches.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. You'll usually see the first input character used as the last target character, so something like this:
```python
y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
```
where `x` is the input batch and `y` is the target batch.

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

> **Exercise:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [12]:
def test():
    x = np.array([[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12]])
    y = np.zeros(x.shape)
    print(x.shape)
    y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
    print(x)
    print(y)
    
test()

(2, 6)
[[ 1  2  3  4  5  6]
 [ 7  8  9 10 11 12]]
[[  2.   3.   4.   5.   6.   1.]
 [  8.   9.  10.  11.  12.   7.]]


In [38]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = n_seqs * n_steps
    n_batches = len(arr) // characters_per_batch
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * characters_per_batch]
    
    # Reshape into n_seqs rows
    arr = arr.reshape(n_seqs, n_batches * n_steps)
    print(arr.shape)
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        print(x.shape)
        # The targets, shifted by one
        y = np.zeros(x.shape)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [80]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

(10, 198500)
(10, 50)


In [40]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[44 72 37 12 25 41 38 54 47 52]
 [54 37  8 54 35 22 25 54 65 22]
 [77  0 35  2 52 52 17 58 41 78]
 [35 54 23 28 38  0 35 65 54 72]
 [54  0 25 54  0 78 50 54 78  0]
 [54 82 25 54 14 37 78 52 22 35]
 [72 41 35 54 19 22  8 41 54 45]
 [ 9 54 60 28 25 54 35 22 14 54]
 [25 54  0 78 35 57 25  2 54  4]
 [54 78 37  0 23 54 25 22 54 72]]

y
 [[ 72.  37.  12.  25.  41.  38.  54.  47.  52.  52.]
 [ 37.   8.  54.  35.  22.  25.  54.  65.  22.   0.]
 [  0.  35.   2.  52.  52.  17.  58.  41.  78.  50.]
 [ 54.  23.  28.  38.   0.  35.  65.  54.  72.   0.]
 [  0.  25.  54.   0.  78.  50.  54.  78.   0.  38.]
 [ 82.  25.  54.  14.  37.  78.  52.  22.  35.  40.]
 [ 41.  35.  54.  19.  22.   8.  41.  54.  45.  22.]
 [ 54.  60.  28.  25.  54.  35.  22.  14.  54.  78.]
 [ 54.   0.  78.  35.  57.  25.   2.  54.   4.  72.]
 [ 78.  37.   0.  23.  54.  25.  22.  54.  72.  41.]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`. This will be a scalar, that is a 0-D tensor. To make a scalar, you create a placeholder without giving it a size.

> **Exercise:** Create the input placeholders in the function below.

In [69]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, (batch_size, num_steps), name='inputs')
    targets = tf.placeholder(tf.int32, (batch_size, num_steps), name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. For example,

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow will create different weight matrices for all `cell` objects. Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

> **Exercise:** Below, implement the `build_lstm` function to create these LSTM cells and the initial state.

In [48]:
print([4] * 3)

[4, 4, 4]


In [73]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    # Use a basic LSTM cell
    lstm =  tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell outputs
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(lstm_size), output_keep_prob=keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character, so we want this layer to have size $C$, the number of classes/characters we have in our text.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$. 

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells. We get the LSTM output as a list, `lstm_output`. First we need to concatenate this whole list into one array with [`tf.concat`](https://www.tensorflow.org/api_docs/python/tf/concat). Then, reshape it (with `tf.reshape`) to size $(M * N) \times L$.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

> **Exercise:** Implement the output layer in the function below.

In [75]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        lstm_output: List of output tensors from the LSTM layer
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # Concatenate lstm_output over axis 1 (the columns)
    seq_output = tf.concat(lstm_output, axis=1)
    # Reshape seq_output to a 2D tensor with lstm_size columns
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        # Create the weight and bias variables here
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

>**Exercise:** Implement the loss calculation in the function below.

In [77]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    # the shape of targets is 
    
    # One-hot encode targets and reshape to match logits, one row per sequence per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped =tf.reshape(y_one_hot, (-1, num_classes))
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [52]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

> **Exercise:** Use the functions you've implemented previously and `tf.nn.dynamic_rnn` to build the network.

In [66]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN with tf.nn.dynamic_rnn 
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss =  build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

## Hyperparameters

Here are the hyperparameters for the network.

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

In [81]:
batch_size = 100         # Sequences per batch
num_steps = 100          # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001    # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

> **Exercise:** Set the hyperparameters above to train the network. Watch the training loss, it should be consistently dropping. Also, I highly advise running this on a GPU.

In [82]:
epochs = 20
# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

(100, 19800)
(100, 100)
Epoch: 1/20...  Training Step: 1...  Training loss: 4.4220...  0.2471 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 2...  Training loss: 4.3362...  0.1860 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 3...  Training loss: 3.8982...  0.1710 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 4...  Training loss: 5.6504...  0.1750 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 5...  Training loss: 4.0969...  0.1720 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 6...  Training loss: 3.8537...  0.1700 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 7...  Training loss: 3.7850...  0.2010 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 8...  Training loss: 3.6548...  0.1780 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 9...  Training loss: 3.5280...  0.1710 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 10...  Training loss: 3.4605...  0.1710 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 11...  Training loss: 3.4004...  0

Epoch: 1/20...  Training Step: 91...  Training loss: 3.1323...  0.1735 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 92...  Training loss: 3.1126...  0.1745 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 93...  Training loss: 3.1177...  0.1735 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 94...  Training loss: 3.1147...  0.1745 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 95...  Training loss: 3.0987...  0.1775 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 96...  Training loss: 3.1080...  0.1985 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 97...  Training loss: 3.1164...  0.1825 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 98...  Training loss: 3.1023...  0.1735 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 99...  Training loss: 3.1087...  0.1770 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 100...  Training loss: 3.0949...  0.1730 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 101...  Training loss: 3.0985...  0.1745 sec/bat

Epoch: 1/20...  Training Step: 181...  Training loss: 2.7013...  0.1790 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 182...  Training loss: 2.6937...  0.1720 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 183...  Training loss: 2.6662...  0.1715 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 184...  Training loss: 2.6882...  0.1860 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 185...  Training loss: 2.7182...  0.2186 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 186...  Training loss: 2.6481...  0.1890 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 187...  Training loss: 2.6420...  0.1795 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 188...  Training loss: 2.6113...  0.1735 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 189...  Training loss: 2.6152...  0.1711 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 190...  Training loss: 2.6230...  0.1706 sec/batch
(100, 100)
Epoch: 1/20...  Training Step: 191...  Training loss: 2.6122...  0.172

Epoch: 2/20...  Training Step: 270...  Training loss: 2.3674...  0.1750 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 271...  Training loss: 2.3565...  0.1700 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 272...  Training loss: 2.3488...  0.1720 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 273...  Training loss: 2.3403...  0.1760 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 274...  Training loss: 2.3885...  0.1710 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 275...  Training loss: 2.3421...  0.1700 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 276...  Training loss: 2.3514...  0.1720 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 277...  Training loss: 2.3258...  0.1720 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 278...  Training loss: 2.3280...  0.1720 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 279...  Training loss: 2.3181...  0.1700 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 280...  Training loss: 2.3497...  0.172

Epoch: 2/20...  Training Step: 359...  Training loss: 2.2112...  0.2161 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 360...  Training loss: 2.1810...  0.1750 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 361...  Training loss: 2.1928...  0.1730 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 362...  Training loss: 2.1897...  0.1700 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 363...  Training loss: 2.1995...  0.1720 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 364...  Training loss: 2.1925...  0.1720 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 365...  Training loss: 2.2096...  0.1720 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 366...  Training loss: 2.2056...  0.1700 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 367...  Training loss: 2.1790...  0.1720 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 368...  Training loss: 2.1728...  0.1720 sec/batch
(100, 100)
Epoch: 2/20...  Training Step: 369...  Training loss: 2.1742...  0.173

...

Epoch: 18/20...  Training Step: 3530...  Training loss: 1.2450...  0.2036 sec/batch
(100, 100)
Epoch: 18/20...  Training Step: 3531...  Training loss: 1.2445...  0.1760 sec/batch
(100, 100)
Epoch: 18/20...  Training Step: 3532...  Training loss: 1.2418...  0.1735 sec/batch
(100, 100)
Epoch: 18/20...  Training Step: 3533...  Training loss: 1.2725...  0.1750 sec/batch
(100, 100)
Epoch: 18/20...  Training Step: 3534...  Training loss: 1.3090...  0.1750 sec/batch
(100, 100)
Epoch: 18/20...  Training Step: 3535...  Training loss: 1.2539...  0.1730 sec/batch
(100, 100)
Epoch: 18/20...  Training Step: 3536...  Training loss: 1.2529...  0.1740 sec/batch
(100, 100)
Epoch: 18/20...  Training Step: 3537...  Training loss: 1.2354...  0.1775 sec/batch
(100, 100)
Epoch: 18/20...  Training Step: 3538...  Training loss: 1.2366...  0.1735 sec/batch
(100, 100)
Epoch: 18/20...  Training Step: 3539...  Training loss: 1.2741...  0.1735 sec/batch
(100, 100)
Epoch: 18/20...  Training Step: 3540...  Training 

Epoch: 19/20...  Training Step: 3618...  Training loss: 1.2525...  0.1760 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3619...  Training loss: 1.2294...  0.1745 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3620...  Training loss: 1.2415...  0.1725 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3621...  Training loss: 1.2488...  0.1740 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3622...  Training loss: 1.2339...  0.1795 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3623...  Training loss: 1.2205...  0.1780 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3624...  Training loss: 1.2829...  0.1742 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3625...  Training loss: 1.2518...  0.1740 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3626...  Training loss: 1.2903...  0.1750 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3627...  Training loss: 1.2696...  0.1745 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3628...  Training 

Epoch: 19/20...  Training Step: 3706...  Training loss: 1.2589...  0.1760 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3707...  Training loss: 1.2269...  0.1740 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3708...  Training loss: 1.2659...  0.1760 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3709...  Training loss: 1.2183...  0.1750 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3710...  Training loss: 1.2635...  0.1735 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3711...  Training loss: 1.2435...  0.1740 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3712...  Training loss: 1.2740...  0.1740 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3713...  Training loss: 1.2680...  0.1750 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3714...  Training loss: 1.2307...  0.1745 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3715...  Training loss: 1.2055...  0.1730 sec/batch
(100, 100)
Epoch: 19/20...  Training Step: 3716...  Training 

Epoch: 20/20...  Training Step: 3794...  Training loss: 1.2086...  0.1745 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3795...  Training loss: 1.2326...  0.1735 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3796...  Training loss: 1.2389...  0.1750 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3797...  Training loss: 1.2195...  0.1735 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3798...  Training loss: 1.2416...  0.1750 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3799...  Training loss: 1.2170...  0.1740 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3800...  Training loss: 1.1931...  0.1740 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3801...  Training loss: 1.1946...  0.1780 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3802...  Training loss: 1.2186...  0.1735 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3803...  Training loss: 1.2020...  0.1750 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3804...  Training 

Epoch: 20/20...  Training Step: 3882...  Training loss: 1.2261...  0.1755 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3883...  Training loss: 1.2298...  0.1745 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3884...  Training loss: 1.1947...  0.1745 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3885...  Training loss: 1.1902...  0.1765 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3886...  Training loss: 1.2222...  0.1735 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3887...  Training loss: 1.2185...  0.1755 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3888...  Training loss: 1.1879...  0.1740 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3889...  Training loss: 1.2233...  0.1755 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3890...  Training loss: 1.2207...  0.1740 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3891...  Training loss: 1.2070...  0.1735 sec/batch
(100, 100)
Epoch: 20/20...  Training Step: 3892...  Training 

#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [83]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints\\i3960_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i3000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i3200_l512.ckpt"
all_mo

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [84]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [85]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [86]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints\\i3960_l512.ckpt'

In [87]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints\i3960_l512.ckpt
Farling. Anna
had an exceptionally settled in the doctors had said in her heart,
the stranger the feelings took his hands and should be a minute. The
conversation they had answered herself that he would have taken her hand. His
eyes stould show they walked off the palached conversation when they had
said, that there was the steps of love with the princess, sitting on
the people the patashalisy, her soul of the conversation and heating
was her sister. Alexey Alexandrovitch's facult and satts of complate. The
carriage had already for a stole and the promise before in this table.
He was saying that she had been in the peasants who had so mented
the post of the person who should have said so what had been triede
and sooner and merely he was not the sense of a party of the part of the
strong there and mortification and askayed and the conversation of
the same way of their most comes in the countrily of the consideration,


In [88]:
checkpoint = 'checkpoints/i200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i200_l512.ckpt
Far hee, an so tot th seesse one, bit as her to ton tha ate ar on onte ante ant hhe see so tothe hose an an ont hime tin as tores ate the teres oude an oth tor sis aos tot ont oud thang
tout he sasin on se sithas wan other, whe sos tite he te toser ou on to and onthe se and tot his ter has tit ho wotes ot har tos the thand
ses ans ho whas hed she tere thint ho see sores he whe sat oot as oot orase sotas anded ane thet we te ant he whing te tin tosse hes ared to tha te ho ad ante on tethar the sad sont int he win he ath atis and at her sis onte wons whes and th sar hes th ar sos han sorer serese set it hors an ons heer setind the shime ton titon he wint ther thees and tet ane tin he whes
han the ad ant the who sing tor th ass aod an iog wit an ser ote hat ho hes so sot ald ones on at te an herer ad atot thas an hitis
time tans aod and ther as athing ous on eo he setin han the thers ou thes ton ared
te hit the whind sit

In [89]:
checkpoint = 'checkpoints/i600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i600_l512.ckpt
Farnce with, will as the carriander, and her whot had
his wat how hardion, and her what wish his becanting had sought of soiching the san the wourd her sene to her
abratian antount and a comesteling
and him and said of has stand, as wat offres it the sterted the somestade thit who had the sadion horself
of aspance, had and storg that had thin when te soud and whone that her herd at
and hould as the her to the some wat hearing wat
seild not and a tort ansters wore of her heard bof thit his shere hord toming whe houdd had te ther thas histored a the had say as her ald and him. The passing,
and the sorpt the plosion the sailicustions anounter to shand her and
sare wat her
ast to her and ale the him he conct andisiont the wand and showed his tomere the conter, and her, as tring would her hered har and woul say she hoded, and the
herstors a thomesh andiring the was of the sanding to diget of
the was as the hissed and he sa

In [90]:
checkpoint = 'checkpoints/i1200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1200_l512.ckpt
Fardy."



Ant of the princess and the position to attacking houres of her, he had not all
the princapliente had not took him and shilk to the seet in the stares
which he could not couls to be all and she was an heart to the
might, and the more to hers to some thinks of the paining had been that it
with the house of the hind the cromes would say harr time her sore
streation of to
be some so to him as a coldicain works of the still
was the starrion. He handed been a carried herself.

She was to anything of this wood to a go and take his for the sames,
and and all the metten as the courte was heart to to tell what he
seamed, a speckanes on his from that. Time said at his
storts, and seemed to his fane in which his for hors to her
east of home, world all the stair, though, stend them, but as all the
world which the pasters. The poriter as the said he had brought a centracanious
of the seemons and the charmier, and the c